In [6]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html
import pandas as pd


# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret='C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624',
)



Version 7.8.0 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [7]:
print(reddit.read_only)

True


In [8]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Test post
spoilers
Join the Blackhawk Defense Force – We Need You! [MILSIM]
Test
test img 2
Test
Tryna see how this emoji shows
test
Ok so since somebody told me to come here I’m coming here


In [9]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)

wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [66]:

stockCSV = pd.read_csv('stock_info.csv') #Read the CSV fil

stockCSV['Ticker'] = '$' + stockCSV['Ticker'] #Add a $ sign in front of the ticker
stockCSV.head() #Display the first 5 rows of the dataframe


,Ticker,Name,Exchange
0,$A,Agilent Technologies,NYSE
1,$AA,Alcoa Inc.,NYSE
2,$AAN,Aaron's Inc,NYSE
3,$AAT,American Assets Trust,NYSE
4,$AAV,Advantage Oil & Gas Ltd,NYSE


In [97]:

postData = []
import nltk  #Natural Language Toolkit
nltk.download('punkt') #sentence word tokenziation
import textstat #Ta simple python library to calculate statistics from text 
from textblob import TextBlob
import datetime
buySell = 0
# Fetch the top 1000 hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=1000):
        textWords = nltk.word_tokenize(submission.selftext) #tokenize the post text
        wordsPost = submission.selftext.split() #split the title into words
        blob = TextBlob(submission.selftext) #create a textblob object
        
        #determining buy sell depending on 
        if(blob.sentiment.polarity >0):
            buySell = 1
        elif(blob.sentiment.polarity <0):
            buySell = -1
        else:
            buySell = 0
        
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        elif any(word in stockCSV['Ticker'].values for word in wordsPost): #check if any of the words in the title are stock abbreviations
            postData.append({
                 #These are indentifying data
                'stock abbreviation': next(word for word in wordsPost if word in stockCSV['Ticker'].values),
                'stock name': stockCSV.loc[stockCSV['Ticker'] == next(word for word in wordsPost if word in stockCSV['Ticker'].values)]['Name'].values[0],
                'title': submission.title,
                'date': datetime.datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'id': submission.id,
                'url': submission.url, 
                'post text': submission.selftext, #post text
                #All of the following are features that can be used to predict the stock price
                'upvote Ratio': submission.upvote_ratio, #upvote ratio
                'comment number': submission.num_comments, #number of comments
                'text length': len(submission.selftext),#length of the post text
                'flesh_kincaid_grade': textstat.flesch_kincaid_grade(submission.selftext), #Flesch Kincaid Grade
                'lexical_diversity': len(set(textWords)) / len(textWords), #lexical diversity
                'Sentiment (polarity):': blob.sentiment.polarity, #Sentiment
                'Sentiment (subjectivity)': blob.sentiment.subjectivity, #Subjectivity
                'Buy/Sell': buySell #Buy or Sell
            })
        else:
            continue
        

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.DataFrame(postData)
df 

,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesh_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell
0,$TSLA,Tesla Motors,Thank you Elon! $TSLA $10k to $149k in under a...,2024-11-11 18:28:35,1gp6l38,https://i.redd.it/k519nqm4xc0e1.jpeg,Opened a lotto account last week after the ele...,0.84,201,129,4.0,0.884615,0.175000,0.308333,1
1,$MSTR,Microstrategy Inc.,6 years of trading to break even…,2024-11-11 10:02:25,1gou6tr,https://i.redd.it/1qpznfqtea0e1.jpeg,I literally lost money for 5 years and decided...,0.97,188,217,7.3,0.840909,-0.025000,0.241667,-1
2,$INTC,Intel Corp.,Intel CEO optimistic about CHIPS Act’s future ...,2024-11-09 06:40:01,1gn8a0p,https://www.reddit.com/r/wallstreetbets/commen...,[https://www.oregonlive.com/silicon-forest/202...,0.94,115,541,33.1,0.846154,0.000000,0.100000,0
3,$INTC,Intel Corp.,Biden Admin Rushing to Distribute CHIPS Act Fu...,2024-11-08 17:59:36,1gmvo4c,https://www.bloomberg.com/news/articles/2024-1...,$INTC,0.96,564,5,-3.5,1.000000,0.000000,0.000000,0
4,$OSTK,Overstock.Com,"Good run for $COIN, but when to get out?",2024-11-09 00:43:56,1gn3b1m,https://www.reddit.com/r/wallstreetbets/commen...,With $COIN fundamentals pointing firmly down a...,0.70,45,1515,10.6,0.566154,0.054167,0.466782,1
5,$PBI,Pitney Bowes Inc,PBI YOLO 11/7,2024-11-07 21:52:23,1gm8kco,https://www.reddit.com/r/wallstreetbets/commen...,"Listen up, degenerates. Pitney Bowes (PBI) is ...",0.92,56,1276,6.9,0.588028,0.060440,0.446337,1
6,$TSLA,Tesla Motors,"Thanks Papa Elon, Trump",2024-11-06 09:55:55,1gl0cru,https://www.reddit.com/gallery/1gl0cru,Been in $TSLA leaps since July. Decided to ope...,0.68,248,177,1.8,0.682927,0.025000,0.250000,1
7,$TSLA,Tesla Motors,Another $TSLA election bet,2024-11-06 15:07:27,1gl7qqr,https://i.redd.it/bvw8hvro8czd1.jpeg,I figured $TSLA calls would either go to the m...,0.89,16,194,3.5,0.928571,0.361905,0.345238,1
8,$SMCI,Super Micro Computer,How fucked am I ?,2024-11-06 22:50:25,1glhn5l,https://i.redd.it/iwq8zedajezd1.jpeg,I have a bunch of $SMCI puts and the theta is ...,0.55,19,401,4.0,0.829545,-0.077778,0.394444,-1
9,$NVMI,Nova Measuring Instruments Ltd,Earnings monkey see earnings monkey do,2024-11-06 13:35:29,1gl5ir4,https://i.redd.it/hnilc92asbzd1.jpeg,All in for $NVMI earnings tomorrow. New memory...,0.69,10,134,5.0,0.884615,0.156566,0.359848,1


In [ ]:
#TO DO 
#1. Add Data from Yahoo Finance API to the dataframe
#2. Get the "buy/sell" value of that stock for the day
#3. See what happended a day later
#4. Get the Delta between the two values
#5. Use that to create a model that predicts the stock price